# "Concevez une application au service de la santé publique"
_Cleaning Notebook_
13/08 16h

## 0 Preliminaries

In [0]:
# # Creating/activating virtual environment and pipfile if not existing
# !pipenv install os
# !pip shell

### 0.0 Importing Packages and Modules

In [0]:
# # Importing requirements.txt file
# from google.colab import files
# files.upload()

In [0]:
# # Installing required packages
# !pipenv install

In [4]:
import sys
# Checking whether the notebook is on Colab or PC
is_colab = 'google.colab' in sys.modules
is_colab, sys.executable

(True, '/usr/bin/python3')

In [5]:
# Replaced by 
# !pip install -r requirements.txt
# Installations and importations required for Google Colab version
if is_colab==True:
  from google.colab import files, output
  !pip install python-stdnum
  !pip install gTTS
  !pip install ipywidgets
else:
    pass

     |████████████████████████████████| 849kB 4.8MB/s 
  Created wheel for gtts-token: filename=gTTS_token-1.1.3-cp36-none-any.whl size=4097 sha256=ec2b752de35ff14ec8430213fdd00ef7b799bab748a39724ac6e0e1d5c3d6c6c
  Stored in directory: /root/.cache/pip/wheels/dd/11/61/33f7e51bf545e910552b2255eead2a7cd8ef54064b46dceb34
Successfully built gtts-token


In [0]:
## IMPORTATIONS
import os
import io
import json
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.facecolor']='w'
from stdnum.exceptions import *
from stdnum.util import *
from stdnum.ean import *
from ipywidgets import interact, interact_manual

In [0]:
# Function to display small dataframes side by side
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),
                 raw=True)

In [0]:
# Setting pandas display options 
dictPdSettings = {'display.max_rows': 500, 'display.width': 100,
                    'display.max_colwidth': 20,
                    'display.float_format': lambda x: '%.0f' % x}
for k,v in dictPdSettings.items():
  pd.set_option(k,v)

In [0]:
# To play audio text-to-speech during execution
from IPython.display import Audio
from gtts import gTTS

def speak(text):
    with io.BytesIO() as f:
        gTTS(text=text, lang='en').write_to_fp(f)
        f.seek(0)
        return Audio(f.read(), autoplay=True)

In [10]:
speak('Packages and modules successfully imported')

### 0.1 Importing a sample of the dataset

In order to facilitate further manipulation of the dataset, we only upload a sample of the dataset at fisrt.

This will allow us to determine the best encoding form for each column therefore resulting in a better optimization of memory usage while uploading the full dataset.

In [11]:
p = 0.005  # reads 0.5% of the lines

if is_colab==False:
  # Importing database from the current Colab content directory
  df = pd.read_csv('../DATA/en.openfoodfacts.org.products.csv',
         header=0, sep='\t', low_memory=False, encoding ='utf-8',
         skiprows=lambda i: i>0 and np.random.random() > p)
  print("Imported from PC")
else:
  # Importing database from the Internet if not already in Colab environment
  if not os.path.exists('en.openfoodfacts.org.products.csv'):
    !wget https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv
    print("Imported from the Internet in colab environment")
  else:
    pass
  df = pd.read_csv('/content/en.openfoodfacts.org.products.csv',
         header=0, sep='\t', low_memory=False, encoding ='utf-8',
         skiprows=lambda i: i>0 and np.random.random() > p)
  print("Imported from colab environment")

--2020-05-04 13:12:32--  https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv
Resolving static.openfoodfacts.org (static.openfoodfacts.org)... 213.36.253.206
Connecting to static.openfoodfacts.org (static.openfoodfacts.org)|213.36.253.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2854379836 (2.7G) [text/csv]
Saving to: ‘en.openfoodfacts.org.products.csv’

en.openfoodfacts.or 100%[===================>]   2.66G  19.6MB/s    in 2m 8s   

2020-05-04 13:14:41 (21.3 MB/s) - ‘en.openfoodfacts.org.products.csv’ saved [2854379836/2854379836]

Imported from the Internet in colab environment
Imported from colab environment


In [12]:
speak('Dataset successfully imported')

### 0.2 Memory usage reduction

In order to facilitate the manipulation, we change the format of encoding
- ints and floats are to be downcasted
- objects are to be changed to category (if the number of unique values is less tha, half the number of rows)

NB: If we already know the format of data, we can specify the format of each column while loading the data in a pandas dataframe: *pd.read_csv('path', dtype=new_dtypes)*

In [13]:
## Changing format of object and float64 columns routine

new_dtypes = {}

# object -> category (if nb unique < half total nb of rows and not XXX_100g)
for col in df.columns.values:
    if df[col].dtype == 'object' :
      if len(df[col].unique()) / len(df[col]) < 0.5:
        new_dtypes[col] = 'category'
      else:
        new_dtypes[col] = 'object'

# float64 -> float32
for col in df.columns.values:
    if df[col].dtype == 'float64':
      if df[col].notna().sum()!=0: # 1% sample columns happening to be all null 
        new_dtypes[col] = 'float32'

# int64 -> int8
for col in df.columns.values:
    if df[col].dtype == 'int64':
        new_dtypes[col] = 'int8'
        
print(new_dtypes)

{'code': 'object', 'url': 'object', 'creator': 'category', 'created_datetime': 'object', 'last_modified_datetime': 'object', 'product_name': 'object', 'generic_name': 'category', 'quantity': 'category', 'packaging': 'category', 'packaging_tags': 'category', 'brands': 'category', 'brands_tags': 'category', 'categories': 'category', 'categories_tags': 'category', 'categories_en': 'category', 'origins': 'category', 'origins_tags': 'category', 'manufacturing_places': 'category', 'manufacturing_places_tags': 'category', 'labels': 'category', 'labels_tags': 'category', 'labels_en': 'category', 'emb_codes': 'category', 'emb_codes_tags': 'category', 'first_packaging_code_geo': 'category', 'cities_tags': 'category', 'purchase_places': 'category', 'stores': 'category', 'countries': 'category', 'countries_tags': 'category', 'countries_en': 'category', 'ingredients_text': 'category', 'allergens': 'category', 'traces': 'category', 'traces_tags': 'category', 'traces_en': 'category', 'serving_size': 

In [14]:
# Importing the whole dataframe with optimized types

if is_colab==False:
  # Importing database from my current local Git Repo
  df = pd.read_csv('../DATA/en.openfoodfacts.org.products.csv', dtype=new_dtypes,
         header=0, sep='\t', low_memory=False, encoding ='utf-8')
  print("Imported from PC")
else:
  df = pd.read_csv('/content/en.openfoodfacts.org.products.csv', dtype=new_dtypes,
         header=0, sep='\t', low_memory=False, encoding ='utf-8')
  print("Imported from colab environment")

Imported from colab environment


In [15]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1359936 entries, 0 to 1359935
Columns: 181 entries, code to carnitine_100g
dtypes: category(49), float32(77), float64(45), int8(2), object(8)
memory usage: 2.7 GB


In [16]:
speak("Reduction of memory usage done")

Main conclusions : 
- Description :
  954463 rows and 175 columns
  some of them categorical, 
  some of them numerical
- Memory usage reduction : 
  6.0 GB -> 2.2 GB

### 0.3 First Overview

In [17]:
# Shape of the dataset
df_sh = df.shape # (1173417, 178) (954463, 175)
df_sh

(1359936, 181)

In [18]:
# General description 
desc = df.describe(include='all')

# Types and content of the columns
desc_type = pd.DataFrame(df.dtypes, columns=['type']).T
nb_Null = (df_sh[0]-desc.loc["count"].T)
pct_null = (df_sh[0]-desc.loc["count"].T)*100/df_sh[0]
desc_null = pd.DataFrame([nb_Null, pct_null], index=['nb null','pct null'])

# Concatenate general, type and null stats
desc = pd.concat([desc_type, desc_null, desc])
desc

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,...,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
type,object,object,category,int8,object,int8,object,object,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,category,float64,category,category,category,category,category,category,category,category,float64,category,category,category,...,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float64,float32,float32,float64,float32,float32,float32,float64,float32,float32,float32,float64,float64,float64,float32,float32,float64,float32,float32
nb null,0,0,4,0,0,0,0,59561,1258675,975976,1145236,1145245,594280,594329,648776,648779,648779,1300436,1300535,1265197,1265234,1040041,1040019,1040019,1269537,1269564,1300066,1359936,1295427,1226771,1162335,2109,2111,2111,738903,1199698,1359936,1285121,1264229,1264229,...,1351512,1347988,1359042,1354207,1359832,1359609,1268797,1359339,1092056,1346631,1096385,1345813,1350104,1355793,1356029,1359661,1357585,1359792,1359736,1358021,1359583,1359814,1359786,1353914,1359648,1348835,1359642,1354730,1359934,1359521,1349789,801840,1359892,1359933,1359936,1359899,1358228,1359913,1359897,1359919
pct null,0,0,0,0,0,0,0,4,93,72,84,84,44,44,48,48,48,96,96,93,93,76,76,76,93,93,96,100,95,90,85,0,0,0,54,88,100,94,93,93,...,99,99,100,100,100,100,93,100,80,99,81,99,99,100,100,100,100,100,100,100,100,100,100,100,100,99,100,100,100,100,99,59,100,100,100,100,100,100,100,100
count,1359936,1359936,1359932,1359936,1359936,1359936,1359936,1300375,101261,383960,214700,214691,765656,765607,711160,711157,711157,59500,59401,94739,94702,319895,319917,319917,90399,90372,59870,0,64509,133165,197601,1357827,1357825,1357825,621033,160238,0,74815,95707,95707,...,8424,11948,894,5729,104,327,91139,597,267880,13305,263551,14123,9832,4143,3907,275,2351,144,200,1915,353,122,150,6022,288,11101,294,5206,2,415,10147,558096,44,3,0,37,1708,23,39,17
unique,1359696,1359774,10936,NaN,1113311,NaN,975865,877895,74281,31137,37468,30317,132588,104994,90428,54121,54121,10261,9055,21504,19451,58070,43226,43226,24831,22082,3353,NaN,6208,9785,11761,8003,2928,2928,515747,5744,NaN,8864,8946,8946,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,8006890683851,http://world-en....,kiliweb,NaN,2020-04-23T17:22...,NaN,2019-03-16T21:14...,Aceite de oliva ...,Ice cream,500 g,"sachet,plastique","sachet,plastique",Carrefour,carrefour,Snacks,en:snacks,Snacks,France,france,France,france,en:gluten-free,en:organic,Organic,FR 72.264.002 EC,fr-72-264-002-ec,"47.833333,-0.333333",NaN,sable-sur-sarthe...,France,Carrefour,France,en:france,France,

In [19]:
print(tuple(df.columns.values))

('code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_en', 'origins', 'origins_tags', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_en', 'ingredients_text', 'allergens', 'allergens_en', 'traces', 'traces_tags', 'traces_en', 'serving_size', 'serving_quantity', 'no_nutriments', 'additives_n', 'additives', 'additives_tags', 'additives_en', 'ingredients_from_palm_oil_n', 'ingredients_from_palm_oil', 'ingredients_from_palm_oil_tags', 'ingredients_that_may_be_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil', 'ingredients_that_may_be_from_palm_oil_tags', 'nutriscore_score', 'nutriscore_grade', '

14 columns with no entry, no row has all the columns filled.

In [20]:
speak('Description overview done')

Main conclusions : 

## 1 Dropping columns and rows

### 1.1 Filtering out non relevant columns
Some columns can immediately be classified as non useful for our project and therefore dropped : 
- 5 columns concerning to the image of the product: 'image_small_url',	'image_ingredients_url',	'image_ingredients_small_url',	'image_nutrition_url'	'image_nutrition_small_url'
- 3 columns concerning the level of completion of the row: 'states', 'states_tags', 'states_en' 


In [21]:
li_col_del = ['image_url', 'image_small_url',	'image_ingredients_url',
              'image_ingredients_small_url',	'image_nutrition_url',
              'image_nutrition_small_url',
              'states', 'states_tags', 'states_en' ]
# Filtering out 8 non relevant columns
df.drop(columns=li_col_del, inplace=True)
df.shape

(1359936, 172)

### 1.2 Almost empty columns

Let's see how the columns are filled.


In [22]:
# Checking nb of not null values in the columns
nb_not_null = pd.DataFrame((~df.isna()).sum(axis =0), columns=['nb'])
nb_not_null.sort_values(by=['nb'], axis=0, ascending=True, inplace=True)
nb_not_null.T.head(150)

,ingredients_from_palm_oil,-mead-acid_100g,-gondoic-acid_100g,-elaidic-acid_100g,-melissic-acid_100g,cities,-lignoceric-acid_100g,-nervonic-acid_100g,-erucic-acid_100g,allergens_en,-caproic-acid_100g,water-hardness_100g,no_nutriments,ingredients_that_may_be_from_palm_oil,-caprylic-acid_100g,-myristic-acid_100g,-stearic-acid_100g,-dihomo-gamma-linolenic-acid_100g,-butyric-acid_100g,chlorophyl_100g,-montanic-acid_100g,glycemic-index_100g,-palmitic-acid_100g,-cerotic-acid_100g,-capric-acid_100g,-gamma-linolenic-acid_100g,-behenic-acid_100g,-maltose_100g,-lauric-acid_100g,nucleotides_100g,carnitine_100g,-maltodextrins_100g,-arachidic-acid_100g,-oleic-acid_100g,beta-glucan_100g,additives,-arachidonic-acid_100g,serum-proteins_100g,choline_100g,inositol_100g,...,serving_size,serving_quantity,fiber_100g,nova_group,nutriscore_grade,nutriscore_score,nutrition-score-fr_100g,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,additives_n,ingredients_text,categories_tags,categories_en,main_category,main_category_en,categories,brands_tags,brands,energy-kcal_100g,saturated-fat_100g,sugars_100g,sodium_100g,salt_100g,carbohydrates_100g,fat_100g,proteins_100g,energy_100g,product_name,pnns_groups_1,countries_en,countries_tags,countries,pnns_groups_2,creator,last_modified_datetime,last_modified_t,created_datetime,created_t,url,code
nb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,2,2,2,3,3,3,4,5,7,9,9,13,17,17,22,22,23,29,30,36,37,39,...,432735,432967,436186,534308,558087,558087,558096,621033,621033,621033,621033,711157,711157,711157,711157,711160,765607,765656,1029574,1046855,1073606,1079698,1079706,1094156,1094701,1096120,1101796,1300375,1346460,1357825,1357825,1357827,1359538,1359932,1359936,1359936,1359936,1359936,1359936,1359936


In [23]:
# Displaying a table with bins
mask = pd.cut(nb_not_null['nb'], [-1,0,5,10,100,300,1000000]) # 
mask.value_counts(normalize=False, sort=False)

(-1, 0]           14
(0, 5]            12
(5, 10]            3
(10, 100]         17
(100, 300]        12
(300, 1000000]    92
Name: nb, dtype: int64

In [24]:
# Plotting nb of not null per column
@interact_manual
def plot_more_than_n_notna(n =(1,1000,50)):
   ax = nb_not_null['nb'][nb_not_null['nb']>n].reset_index()\
                .plot.bar(x='index', y='nb', figsize=(25,4))
   ax.set(yscale='log')
   return ax

interactive(children=(IntSlider(value=451, description='n', max=1000, min=1, step=50), Button(description='Run…

In [25]:
# Displaying names of columns with less than 300 entries
mask = pd.cut(nb_not_null['nb'], [-1,300])
li_col_del = list(mask[~mask.values.isna()].index)
print(len(li_col_del))
for s in li_col_del:
  print(s, end='|')

58
ingredients_from_palm_oil|-mead-acid_100g|-gondoic-acid_100g|-elaidic-acid_100g|-melissic-acid_100g|cities|-lignoceric-acid_100g|-nervonic-acid_100g|-erucic-acid_100g|allergens_en|-caproic-acid_100g|water-hardness_100g|no_nutriments|ingredients_that_may_be_from_palm_oil|-caprylic-acid_100g|-myristic-acid_100g|-stearic-acid_100g|-dihomo-gamma-linolenic-acid_100g|-butyric-acid_100g|chlorophyl_100g|-montanic-acid_100g|glycemic-index_100g|-palmitic-acid_100g|-cerotic-acid_100g|-capric-acid_100g|-gamma-linolenic-acid_100g|-behenic-acid_100g|-maltose_100g|-lauric-acid_100g|nucleotides_100g|carnitine_100g|-maltodextrins_100g|-arachidic-acid_100g|-oleic-acid_100g|beta-glucan_100g|additives|-arachidonic-acid_100g|serum-proteins_100g|choline_100g|inositol_100g|casein_100g|nutrition-score-uk_100g|omega-9-fat_100g|-glucose_100g|beta-carotene_100g|-fructose_100g|-eicosapentaenoic-acid_100g|silica_100g|-sucrose_100g|taurine_100g|-docosahexaenoic-acid_100g|chromium_100g|ph_100g|molybdenum_100g|-li

In [26]:
# Printing remaining columns
li_rem_col = [s for s in list(df.columns) if s not in li_col_del]
print(len(li_col_del))
for s in li_rem_col:
  print(s, end='|')

58
code|url|creator|created_t|created_datetime|last_modified_t|last_modified_datetime|product_name|generic_name|quantity|packaging|packaging_tags|brands|brands_tags|categories|categories_tags|categories_en|origins|origins_tags|manufacturing_places|manufacturing_places_tags|labels|labels_tags|labels_en|emb_codes|emb_codes_tags|first_packaging_code_geo|cities_tags|purchase_places|stores|countries|countries_tags|countries_en|ingredients_text|allergens|traces|traces_tags|traces_en|serving_size|serving_quantity|additives_n|additives_tags|additives_en|ingredients_from_palm_oil_n|ingredients_from_palm_oil_tags|ingredients_that_may_be_from_palm_oil_n|ingredients_that_may_be_from_palm_oil_tags|nutriscore_score|nutriscore_grade|nova_group|pnns_groups_1|pnns_groups_2|brand_owner|main_category|main_category_en|energy-kj_100g|energy-kcal_100g|energy_100g|energy-from-fat_100g|fat_100g|saturated-fat_100g|monounsaturated-fat_100g|polyunsaturated-fat_100g|omega-3-fat_100g|-alpha-linolenic-acid_100g|ome

We keep columns with more than 300 values except:
- 'carbon-footprint_100g', 'carbon-footprint-from-meat-or-fish_100g',
'fruits-vegetables-nuts_100g', 'fruits-vegetables-nuts-estimate_100g', as they would require more data to be exploitable.

In [27]:
# Adding 4 columns from the "to-be-deleted" columns list 
li_col_del += ['carbon-footprint_100g',
               'carbon-footprint-from-meat-or-fish_100g',
               'fruits-vegetables-nuts_100g',
               'fruits-vegetables-nuts-estimate_100g']
print(f'nb of columns to drop: {len(li_col_del)}')

nb of columns to drop: 62


In [28]:
print("shape before", df.shape)
df.drop(columns=li_col_del, inplace=True)
print("shape after", df.shape)

shape before (1359936, 172)
shape after (1359936, 110)


We rearrange the columns with similar contents at the same place:

In [0]:
# Changing the name of some of the columns
df.rename(columns={'nutriscore_score': 'nutriscore',
                       'nutriscore_grade': 'nutrigrade',
                       'pnns_groups_1': 'pnns_gp_1',
                       'pnns_groups_2': 'pnns_gp_2',
                       'nutrition-score-fr_100g':'nutriscore-fr'},
              inplace=True)

In [0]:
# Function that moves similar names in an Index at the same place
def move_cat_containing(my_index, strings, order):

  idx_sel = []

  if order == 'last':
    index = my_index
  elif order == 'first':
    index = my_index[::-1]
  else:
    print("--- WARNING : index unchanged.\n -- Wrong order passed. Pass 'first' or 'last'")
    return my_index

  for s in strings:
    idx_sel += [i for i,x in enumerate(index)\
             if x in index[index.str.contains(s)]]
  to_move = index[idx_sel]
  rank = max(idx_sel)
  mod_index = index.drop(to_move)
  for c in to_move:
    mod_index = mod_index.insert(rank,c)
  
  return mod_index if order=='last' else mod_index[::-1]

In [31]:
# Moving 'nutriscores', 'categories', info related to brand and to quantities
cols = df.columns
cols = move_cat_containing(cols,['name'], 'first')
cols = move_cat_containing(cols,['brand'], 'first')
cols = move_cat_containing(cols,['categ','pnns', 'label'], 'first')
cols = move_cat_containing(cols,['nutri', 'nova'], 'first')
cols = move_cat_containing(cols,['_100g'], 'last')
print(tuple(cols))

('code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'main_category_en', 'main_category', 'categories_en', 'categories_tags', 'categories', 'pnns_gp_2', 'pnns_gp_1', 'labels_en', 'labels_tags', 'labels', 'generic_name', 'product_name', 'last_modified_datetime', 'quantity', 'brand_owner', 'brands_tags', 'brands', 'packaging', 'packaging_tags', 'origins', 'origins_tags', 'manufacturing_places', 'manufacturing_places_tags', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_en', 'ingredients_text', 'allergens', 'traces', 'traces_tags', 'traces_en', 'serving_size', 'serving_quantity', 'additives_n', 'additives_tags', 'additives_en', 'ingredients_from_palm_oil_n', 'nutriscore-fr', 'nutrigrade', 'nutriscore', 'nova_group', 'ingredients_from_palm_oil_tags', 'ingredients_that_may_be_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil_tags', 'energy-kj_100g', 'energy-kcal

In [32]:
df = df.reindex(columns=cols)
df.head(1)

,code,url,creator,created_t,created_datetime,last_modified_t,main_category_en,main_category,categories_en,categories_tags,categories,pnns_gp_2,pnns_gp_1,labels_en,labels_tags,labels,generic_name,product_name,last_modified_datetime,quantity,brand_owner,brands_tags,brands,packaging,packaging_tags,origins,origins_tags,manufacturing_places,manufacturing_places_tags,emb_codes,emb_codes_tags,first_packaging_code_geo,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,...,sugars_100g,-lactose_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,selenium_100g,iodine_100g,caffeine_100g,cocoa_100g,phylloquinone_100g
0,0000000000017,http://world-en....,kiliweb,8,2018-06-15T10:38...,-90,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,Vitória crackers,2019-06-25T11:55...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,...,15,nan,nan,nan,nan,nan,nan,8,1,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


### 1.3 Columns with redundant information

Some of the columns seem to contain similar information. For example : 
  - 'created_t' vs.	'created_datetime'
  - 'last_modified_t' vs.	'last_modified_datetime'
  - 'brands' vs.	'brands_tags'
  - 'categories	' vs.	'categories_tags'  vs.	'categories_en'  
  - 'origins' vs.	'origins_tags'
  - 'manufacturing_places' vs.	'manufacturing_places_tags'
  - 'labels' vs.	'labels_tags'  vs.	'labels_en' 
  - 'emb_codes' vs.	'emb_codes_tags'
  - 'countries' vs.	'countries_tags'  vs.	'countries_en'
  - 'traces' vs.	'traces_tags'  vs.	'traces_en'
  - 'additives_n' vs. 'additives' vs.	'additives_tags'  vs.	'additives_en'


Let's see if some of them can be dropped or their content gathered in only one column.
To that end, we define a function '**comp_df**' that :
- compares the content of two columns col1 and col2 of the dataframe
- prints the number or rows with 1) both columns, 2) col1 only, 3) col2 only, and 4) none of them filled
- returns indexes of 1) data in both columns, 2) data only in col1, 3) data only in col2, and 4) no data at all.

After that, we check and compare the content of the columns when both are filled in order to get familiar with the content of the database and determine the columns that can be dropped.

In [0]:
def comp_df (df, col1, col2, print_Y_N):
  m_both = (~df[[col1,col2]].isna()).sum(axis=1)==2 # mask for both cells filled
  m_one = (~df[[col1,col2]].isna()).sum(axis=1)==1 # mask for only one cell filled
  m_col1 = m_one & ~df[col1].isna() # mask for only col1 cell filled
  m_col2 = m_one & ~df[col2].isna() # mask for only col2 cell filled
  mnone = ~m_one & ~m_both
  ind_both = df.index[m_both].to_list()
  ind_col1 = df.index[m_col1].to_list()
  ind_col2 = df.index[m_col2].to_list()
  ind_none = df.index[mnone].to_list()
  if print_Y_N :
    print("nb rows both filled: ", len(ind_both))
    print("nb rows with only", col1, "filled: ", len(ind_col1))
    print("nb rows with only", col2, "filled: ", len(ind_col2))
    print("nb rows not filled: ", len(ind_none))
  else:
    pass
  return(ind_both, ind_col1, ind_col2, ind_none)

In [34]:
# Printing amount of rows in each case:
#(col1 filled, col2 filled, both filled, none filled)
li_col = ['packaging', 'brands', 'categories', 'origins', 'manufacturing_places',
        'labels', 'emb_codes','countries', 'traces']
for c in li_col:
  c1 = c
  c2 = c+'_tags'
  print('COLUMN', c,':')
  t_ind = comp_df(df, c1, c2, True)
  df.loc[t_ind[1], [c1,c2]] = np.nan

COLUMN packaging :
nb rows both filled:  214689
nb rows with only packaging filled:  11
nb rows with only packaging_tags filled:  2
nb rows not filled:  1145234
COLUMN brands :
nb rows both filled:  765599
nb rows with only brands filled:  57
nb rows with only brands_tags filled:  8
nb rows not filled:  594272
COLUMN categories :
nb rows both filled:  711157
nb rows with only categories filled:  3
nb rows with only categories_tags filled:  0
nb rows not filled:  648776
COLUMN origins :
nb rows both filled:  59398
nb rows with only origins filled:  102
nb rows with only origins_tags filled:  3
nb rows not filled:  1300433
COLUMN manufacturing_places :
nb rows both filled:  94702
nb rows with only manufacturing_places filled:  37
nb rows with only manufacturing_places_tags filled:  0
nb rows not filled:  1265197
COLUMN labels :
nb rows both filled:  319877
nb rows with only labels filled:  18
nb rows with only labels_tags filled:  40
nb rows not filled:  1040001
COLUMN emb_codes :
nb row

We design an interactive feature that helps browsing the data and compare the content of XXX AND XXX_tags columns :

In [35]:
@interact_manual
def print_comp_info(col=li_col):
  c1 = col
  c2 = c1+'_tags'
  # Comparing content of 2 columns, print info and returns indexes 
  t_ind = comp_df(df, c1, c2, True)
  # Displaying a random sample of cells:
  '''- when col1 only is filled
  - when col2 only is filled
  - when both columns are filled'''
  display_side_by_side(df.loc[t_ind[1],[c1, c2]]\
                       .sample(5 if len(t_ind[1])>5 else len(t_ind[1])),
                      df.loc[t_ind[2], [c1, c2]]\
                       .sample(5 if len(t_ind[2])>5 else len(t_ind[2])),
                      df.loc[t_ind[0], [c1, c2]]\
                       .sample(5 if len(t_ind[0])>5 else len(t_ind[0])))

interactive(children=(Dropdown(description='col', options=('packaging', 'brands', 'categories', 'origins', 'ma…

After examination of all the columns above-listed, we decide :
- to set values of the 'XXX' column to NaN if not matching with a non NaN value in the 'XXX_tags' column (as we found that they correspond to erroneous values ('?', '/', '-', etc.));
- to drop 'created_t' and 'last_modified_t';
- to drop 'packaging_tags', 'brands_tags', 'categories_tags', 'origins_tags', 'manufacturing_places_tags', 'labels_tags', 'emb_codes_tags', 'countries_tags', 'additives_tags', as the data they contain are redundant with 'XXX' or 'XXX_en' columns;
- to drop 'traces' (instead of 'traces_tags'), as it is less filled then 'traces_tags'
- to drop 'additives' column, as it does not contain enough data (29 entries), that are however in english in 'additives_en'
- to keep 'XXX_en' columns, as they contain useful and sometimes original data;
- to keep 'XXX' columns.

In [0]:
# Setting values of the 'XXX' column to NaN if no value in 'XXX_tag' 
t_ind = comp_df(df, c1, c2, False)
df.loc[t_ind[1], [c1,c2]] = np.nan

In [37]:
# Dropping unuseful columns
li_col_del = ['created_t', 'last_modified_t','packaging_tags',
             'brands_tags', 'categories_tags', 'origins_tags', 
             'manufacturing_places_tags', 'labels_tags',
             'emb_codes_tags','countries_tags', 'traces',
             'additives_tags']
print("shape before", df.shape)
df.drop(columns=li_col_del, inplace=True)
print("shape after", df.shape)

shape before (1359936, 110)
shape after (1359936, 98)


In [38]:
# Checking the remaining columns
print(tuple(df.columns))

('code', 'url', 'creator', 'created_datetime', 'main_category_en', 'main_category', 'categories_en', 'categories', 'pnns_gp_2', 'pnns_gp_1', 'labels_en', 'labels', 'generic_name', 'product_name', 'last_modified_datetime', 'quantity', 'brand_owner', 'brands', 'packaging', 'origins', 'manufacturing_places', 'emb_codes', 'first_packaging_code_geo', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_en', 'ingredients_text', 'allergens', 'traces_tags', 'traces_en', 'serving_size', 'serving_quantity', 'additives_n', 'additives_en', 'ingredients_from_palm_oil_n', 'nutriscore-fr', 'nutrigrade', 'nutriscore', 'nova_group', 'ingredients_from_palm_oil_tags', 'ingredients_that_may_be_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil_tags', 'energy-kj_100g', 'energy-kcal_100g', 'energy_100g', 'energy-from-fat_100g', 'fat_100g', 'saturated-fat_100g', 'monounsaturated-fat_100g', 'polyunsaturated-fat_100g', 'omega-3-fat_100g', '-alpha-linolenic-acid_100g', 'omega-6-fat_100g', '

Some columns are related and provide information useful to retrieve data via feature engineering.
To help automatizing these tasks, we categorize the columns as follows :

**li_cat_fat**: 'saturated_fat_100g', 'monounsaturated_fat_100g', 'polyunsaturated_fat_100g', 'trans-fat_100g', 'cholesterol_100g'

**li_cat_poly_fat**: 'omega-3-fat_100g', 'omega-6-fat_100g'

**li_cat_carb**: 'sugars_100g', 'starch_100g', 'polyols_100g', 'fibers_100g', '-lactose_100g'

**li_cat_comp**: cat_fat, cat_carb, 'proteins_100g'

**li_cat_vit-oligo**:  'vitamin-a_100g',	'vitamin-d_100g',	'vitamin-e_100g',	'vitamin-k_100g',	'vitamin-c_100g',	'vitamin-b1_100g',	'vitamin-b2_100g',	'vitamin-pp_100g',	'vitamin-b6_100g',	'vitamin-b9_100g',	'folates_100g',	'vitamin-b12_100g',	'biotin_100g',	'pantothenic-acid_100g','silica_100g', 'bicarbonate_100g',	'potassium_100g',	'chloride_100g',	'calcium_100g',	'phosphorus_100g',	'iron_100g',	'magnesium_100g',	'zinc_100g',	'copper_100g',	'manganese_100g',	'fluoride_100g', 'selenium_100g, 'chromium_100g', 'molybdenum_100g', 'iodine_100g',	'caffeine_100g', 'taurine_100g', 'cocoa_100g'

In [39]:
speak('Filtered out unimportant columns')

### 1.4 Dropping inoperable rows

We need to get rid of inoperable rows. We will define operability of a row as follows:

- condition 1: the product can be identified, at least by a unique combination of name, brand and quantity ('product_name', 'quantity', 'brands'), OR by a valid and unique barcode ('code')

AND
- condition 2: at least one numerical information is provided i.e. one of the features 'XXX_100g' or one type of nutritional score

#### *Condition 1*:

Let's first check the 'code' column for completion and validity of the data:

In [40]:
# Checking nb of null values in 'code' column
df['code'].isna().sum()

0

No null value in the 'code' column.

We will now create a new column 'code_val' indicating if the barcode is valid.

In [41]:
# Verification of barcodes (EAN-13, EAN-8 and UPC (12-digit) format)
# Creation of a new column : barcode valid or not
df.insert(1,'code_val', df['code'].apply(is_valid))
print("nb of valid codes: ", df['code_val'].sum())
print("nb of invalid codes: ", (~df['code_val']).sum())
print("pctage of valid codes: {:.2f}%".format(df['code_val'].sum()*100/df.shape[0]))

nb of valid codes:  1329965
nb of invalid codes:  29971
pctage of valid codes: 97.80%


When the code is not valid, we will check if the product can be identified by a unique combination of "product_name", "quantity" and "brands". (At least one of the three must be filled).

In [0]:
key_cols = ['product_name', 'quantity', 'brands']

In [0]:
### Cleaning 'product_name' column
# Replacing invalid names (1 or 0 characters) with np.NaN
df.loc[df[df['product_name'].str.len()<2].index,'product_name'] = np.nan

In [0]:
### Cleaning 'quantity' column
df['quantity'] = df['quantity'].astype('object')
# To ensure safe execution of function 'float'
def safe_exe(def_val, function, *args):
    try: return function(*args)
    except: return def_val

# Conversion of string in float if possible, else 0 (accepts 'n1 x n2' string)
def conv_float(my_str):
  idx = 0
  if 'x' in my_str:
    idx = my_str.find('x')
    n1 = safe_exe(0, float, my_str[:idx])
    n2 = safe_exe(0, float, my_str[idx+1:])
    return n1*n2
  else:
    return safe_exe(0, float, my_str)

# Splits string in a list of tuples (number, associated unit)
def num_units(my_str):
  my_str = my_str.lower().strip()
  regex = r'([0-9.,x ]*)\s*([^()0-9 !,\-±=\*\+/.-\?\[\]]*\s*)'
  res = re.findall(regex, my_str)
  res.remove(('', ''))
  num = [conv_float(gr[0].replace(' ','').replace(',','.')) for gr in res]
  unit = [gr[1].strip()  for gr in res]
  res=list(zip(num,unit))
  return num, unit

In [0]:
# Classifying main relevant units in 'mass' and 'volume' units
li_u_mass = ['g', 'kg', 'gr', 'grammes', 'grs','st', 'mg', 'gramm', 'lb','gram',
             'grams', 'gramos', 'lbs', 'gm', 'lt', 'lts','gramme', 'kilo','公克',
             'grammi', 'kgs', 'kgr', 'gms', 'g-', 'grms','pound', 'pounds', 
             'grm', 'grames','غرام', 'جرام','غ', 'غم','جم','g℮', 'г', 'кг', '克', 
             'грамм', 'גרם','kilogramm','gramas','kilogrammae','livres',
             'grame', 'kilos'] 
li_u_vol = ['ml','dl','l','cl', 'oz', 'litre', 'fl', 'litres', 'liter','litro',
            'litri','litr', 'ounces','ounce', 'ltr', 'gallon','half-gallon',
            'litros','litroe', 'liters', 'cc', 'kl', 'pint','pints', 'gal',
            'mls', 'centilitres', 'لتر','مل','ل','ليتر', 'มล', 'ลิตร', 'мл', 'л',
            'litrè', 'milliliter','millilitre', 'γρ', 'литр', 'литра', 'mml',
            'מ״ל','millilitres','λίτρο', 'mĺ', 'cm', 'cm³' ]

In [0]:
# Function selecting the most relevant tuple (num,unit) in the list
# by order of priority : unit in li_prio1, then in li_prio2

def sel_gr(li, li_prio1, li_prio2):
  res = 0
  nums, units = li
  tab_t_u = []
  ind = np.nan
  for i in units:
    if  i in li_prio1:
      tab_t_u.append(2)
    elif  i in li_prio2:
      tab_t_u.append(1)
    else:
      tab_t_u.append(0) if i!='' else tab_t_u.append(np.nan)
  i_tab1 = [i for i in range(len(tab_t_u)) if tab_t_u[i]==2] # indexes of all volumes (prio1)
  i_tab2 = [i for i in range(len(tab_t_u)) if tab_t_u[i]==1] # indexes of all masses (prio2)
  i_tab0 = [i for i in range(len(tab_t_u)) if tab_t_u[i]==0] # indexes of all others (prio3)
  if len(i_tab1)>0: # prio1 (vol)
    ind = i_tab1[np.argmax([nums[i] for i in i_tab1])]
  elif len(i_tab2)>0: # prio2 (mass)
    ind = i_tab2[np.argmax([nums[i] for i in i_tab2])]
  else:
    ind = i_tab0[np.argmax([nums[i] for i in i_tab0])] if len(i_tab0)>0 else np.nan
  return(nums[ind], units[ind]) if ind is not np.nan else (np.nan, np.nan)

In [47]:
# Creating a new database with 'quantity' data (wo null, same index)
df_q = df['quantity'].dropna().to_frame()
print("Nb of notna values in 'quantity': {} on {}, i.e. {:.1f}%"\
           .format(df_q.shape[0], df.shape[0],df_q.shape[0]*100/df.shape[0]))
display(df_q.head(15).T)

Nb of notna values in 'quantity': 383960 on 1359936, i.e. 28.2%


,1,3,4,18,40,50,64,96,99,120,153,157,158,160,162
quantity,130 g,100g,dgesc,80 g,1kg,4 g,1,500 g,120g,Vrac,250 ml,2.46 kg,230g,100 g,60 capsules


In [48]:
# Splitting numerical and unit values in two new columns
df_q['analysis'] = df_q['quantity'].apply(num_units)
df_q[['num_gr','unit_gr']] = pd.DataFrame(df_q['analysis'].tolist(),
                                          index=df_q['analysis'].index)
# Selecting the main (num, unit) as the most relevant tuple from 'analysis'
my_fun = lambda x: sel_gr(x, li_u_vol, li_u_mass)
df_q[['num','unit']] = pd.DataFrame(df_q['analysis'].apply(my_fun).tolist(),
                                    index=df_q['analysis'].index)
df_q.head(15).T

,1,3,4,18,40,50,64,96,99,120,153,157,158,160,162
quantity,130 g,100g,dgesc,80 g,1kg,4 g,1,500 g,120g,Vrac,250 ml,2.46 kg,230g,100 g,60 capsules
analysis,"([130.0], [g])","([100.0], [g])","([0], [dgesc])","([80.0], [g])","([1.0], [kg])","([4.0], [g])","([1.0], [])","([500.0], [g])","([120.0], [g])","([0], [vrac])","([250.0], [ml])","([2.46], [kg])","([230.0], [g])","([100.0], [g])","([60.0], [capsul..."
num_gr,[130.0],[100.0],[0],[80.0],[1.0],[4.0],[1.0],[500.0],[120.0],[0],[250.0],[2.46],[230.0],[100.0],[60.0]
unit_gr,[g],[g],[dgesc],[g],[kg],[g],[],[g],[g],[vrac],[ml],[kg],[g],[g],[capsules]
num,130,100,0,80,1,4,NaN,500,120,0,250,2,230,100,60
unit,g,g,dgesc,g,kg,g,NaN,g,g,vrac,ml,kg,g,g,capsules


In [0]:
# Creating a new column with type of the main unit
d_mass_vol = dict([(u,'mass') if u in li_u_mass else\
                  (u,'vol')  if u in li_u_vol else\
                  (u,'other') for u in df_q['unit'].unique()])
df_q['unit_type'] = df_q['unit'].dropna().map(d_mass_vol)

In [0]:
# Dropping rows where unit or num is null 
df_q = df_q.dropna(subset=['unit', 'num'])
# Dropping rows where num is zero, negative or greater than 10000
df_q = df_q[df_q['num'].between(0.0001,10000)]

In [51]:
df.head(2)

,code,code_val,url,creator,created_datetime,main_category_en,main_category,categories_en,categories,pnns_gp_2,pnns_gp_1,labels_en,labels,generic_name,product_name,last_modified_datetime,quantity,brand_owner,brands,packaging,origins,manufacturing_places,emb_codes,first_packaging_code_geo,cities_tags,purchase_places,stores,countries,countries_en,ingredients_text,allergens,traces_tags,traces_en,serving_size,serving_quantity,additives_n,additives_en,ingredients_from_palm_oil_n,nutriscore-fr,nutrigrade,...,sugars_100g,-lactose_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,selenium_100g,iodine_100g,caffeine_100g,cocoa_100g,phylloquinone_100g
0,0000000000017,True,http://world-en....,kiliweb,2018-06-15T10:38...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,Vitória crackers,2019-06-25T11:55...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,France,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,nan,nan,NaN,...,15,nan,nan,nan,nan,nan,nan,8,1,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,0000000000031,True,http://world-en....,isagoofy,2018-10-13T21:06...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,Cacao,2018-10-13T21:06...,130 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,France,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,nan,nan,NaN,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [52]:
# Merging df_quantity with df, reordering the columns
print("df before: ", df.shape,
      " | df_q: \n", df_q.shape)
df = df.merge(df_q[['num', 'unit','unit_type']],
                      how = 'left',  left_index=True, right_index=True)
df = df.rename(columns = {'num':'quantity_num',
                          'unit':'quantity_unit',
                          'unit_type':'quantity_type'})
df = df.reindex(columns=move_cat_containing(df.columns,['quant'], 'first'))
# Showing df shape and column order
print("df after: \n", df.shape)
for i,c in enumerate(df.columns):
  print('\n'+c, end=' | ') if (i) % 6 == 0  else print(c, end=' | ')

df before:  (1359936, 99)  | df_q: 
 (372944, 7)
df after: 
 (1359936, 102)

code | code_val | url | creator | created_datetime | main_category_en | 
main_category | categories_en | categories | pnns_gp_2 | pnns_gp_1 | labels_en | 
quantity_type | quantity_unit | quantity_num | serving_quantity | quantity | labels | 
generic_name | product_name | last_modified_datetime | brand_owner | brands | packaging | 
origins | manufacturing_places | emb_codes | first_packaging_code_geo | cities_tags | purchase_places | 
stores | countries | countries_en | ingredients_text | allergens | traces_tags | 
traces_en | serving_size | additives_n | additives_en | ingredients_from_palm_oil_n | nutriscore-fr | 
nutrigrade | nutriscore | nova_group | ingredients_from_palm_oil_tags | ingredients_that_may_be_from_palm_oil_n | ingredients_that_may_be_from_palm_oil_tags | 
energy-kj_100g | energy-kcal_100g | energy_100g | energy-from-fat_100g | fat_100g | saturated-fat_100g | 
monounsaturated-fat_100g | polyuns

In [53]:
# Dropping rows with invalid codes AND all key_cols NaN
key_cols = ['product_name', 'quantity_num','quantity_unit', 'brands']
m_wo_code = df['code_val']==False
m_wo_keys = df[key_cols].isna().all(axis=1)
print("nb of rows before:", df.shape[0])
df.drop(index=df[m_wo_code & m_wo_keys].index, inplace=True)
print("nb of rows after:", df.shape[0])

nb of rows before: 1359936
nb of rows after: 1358063


#### *Condition 2*:

Let's now check for the second condition that is "at least one numerical columns filled" among a selection of numerical data OR nutritional score

* Reconstruct 'energy' missing data

Some of the missing values of energy can be assessed if the data of fat, carbohydrate and proteins are available.

In [0]:
def nb_rows_na(df, selcol, how):
  if how == 'all':
    ind = df[df[selcol].isna().all(axis=1)].index
  elif how == 'any':
    ind = df[df[selcol].isna().any(axis=1)].index
  else:
    ind=[]
    print("WARNING: 'how' must be either all or any")
  li_ind = ind.to_list()
  return len(li_ind), li_ind
  
  
def print_nb_rows_na(df, selcol, how):
  print("- columns: ", selcol, "\nnb of null containing rows (" + how + "): ", nb_rows_na(df, selcol, how)[0])

In [93]:
# Number of rows with main numerical data missing
energy_cols = df.columns[df.columns.str.contains('energy')].to_list()
print_nb_rows_na(df, energy_cols, 'all'),
print_nb_rows_na(df, energy_cols, 'any')
main_nut_comp_cols = ['fat_100g','carbohydrates_100g', 'proteins_100g']
print_nb_rows_na(df, main_nut_comp_cols, 'all'),
print_nb_rows_na(df, main_nut_comp_cols, 'any')

- columns:  ['energy-kj_100g', 'energy-kcal_100g', 'energy_100g', 'energy-from-fat_100g'] 
nb of null rows (all):  256338
- columns:  ['energy-kj_100g', 'energy-kcal_100g', 'energy_100g', 'energy-from-fat_100g'] 
nb of null rows (any):  1358060
- columns:  ['fat_100g', 'carbohydrates_100g', 'proteins_100g'] 
nb of null rows (all):  258370
- columns:  ['fat_100g', 'carbohydrates_100g', 'proteins_100g'] 
nb of null rows (any):  267893


In [134]:
# Calculating the number of rows with not ANY energy OR (inclusive)
#  not ANY main nutrit. component filled
ind_no_energy = nb_rows_na(df, energy_cols, 'all')[1]
ind_all_comp_filled = df.drop(index=nb_rows_na(df, main_nut_comp_cols, 'any')[1]).index.to_list()
ind_no_en_AND_main_comp_filled = list(set(ind_no_energy) & set(ind_all_comp_filled))
print('not any energy filled : {}\nall main nutri. components filled : {}'\
      .format(len(ind_no_energy), len(ind_all_comp_filled)))
print('not any energy filled BUT all main nutri comp filled : {}'\
      .format(len(ind_no_en_AND_main_comp_filled)))

not any energy filled : 256338
all main nutri. components filled : 1090170
not any energy filled BUT all main nutri comp filled : 1183


In [0]:
# Reconstruct 'energy-kcal_100g' and 'energy-kj_100g' whenever possible


In [0]:
# Drop if not any energy (256338)

In [132]:
df[ind_no_en_AND_main_comp_filled]

{864256,
 165889,
 26626,
 1339394,
 24581,
 892933,
 851975,
 1062924,
 714773,
 331801,
 866331,
 18460,
 864287,
 1140773,
 1144870,
 874534,
 329768,
 280620,
 401454,
 143413,
 880704,
 333893,
 333894,
 333895,
 333896,
 235701,
 1355857,
 381012,
 383061,
 213079,
 872537,
 1136733,
 1355875,
 1196134,
 227433,
 1146989,
 936047,
 786546,
 878707,
 864372,
 282746,
 247935,
 688259,
 1339240,
 323720,
 47245,
 47246,
 372879,
 47248,
 47249,
 280721,
 399503,
 409741,
 41110,
 961903,
 41112,
 135322,
 47259,
 1339546,
 141472,
 409763,
 1343724,
 47269,
 24743,
 47271,
 47272,
 47273,
 47274,
 47275,
 47276,
 47277,
 16559,
 47278,
 47281,
 47282,
 24755,
 532660,
 32947,
 47283,
 47284,
 1061048,
 47287,
 1071290,
 1071291,
 24762,
 47288,
 47292,
 47293,
 84153,
 47297,
 47298,
 47299,
 24772,
 47300,
 47301,
 24775,
 1138885,
 542921,
 1165508,
 84171,
 84172,
 1165510,
 1165513,
 47311,
 47312,
 47313,
 1163471,
 39123,
 47316,
 47317,
 364756,
 401620,
 24792,
 133336,
 15

In [92]:
(energy_cols.to_list()+main_nut_comp_cols)

['energy-kj_100g',
 'energy-kcal_100g',
 'energy_100g',
 'energy-from-fat_100g',
 'fat_100g',
 'carbohydrates_100g',
 'proteins_100g']

In [102]:
selcol = df.columns[df.columns.str.contains('nova|nutri', regex=True)]
print_nb(df, selcol)

- columns:  Index(['nutriscore-fr', 'nutrigrade', 'nutriscore', 'nova_group'], dtype='object') 
nb of rows all NaN :  666977
- columns:  ['nova_group'] 
nb of rows all NaN :  823320


We drop the 184 115 rows that do not have either one of the following 36 numerical '**XXX_100g**' columns:

['energy_100g'] + li_cat_comp + ['salt_100g'] + li_cat_vit_oligo

However, we keep the data with no nutritional data in **li_cat_nutri** columns, as we are going to try to reconsruct those data later.

In [0]:
# Dropping rows with no numerical data filled among 'cruc_num_col' columns
cruc_num_col = ['energy_100g'] + li_cat_comp + ['salt_100g'] + li_cat_vit_oligo
print("nb of rows before:", df.shape[0])
########
print("nb of rows after:", df.shape[0])

In [0]:
speak('Dropped inoperable rows')

### 1.5 Dealing with duplicates
At first we are going to deal with 'absolute' duplicates (i.e. duplicate on each row). We'll simply keep the first row.


In [186]:
# 1 Keeping the first of duplicates on all columns
print("before", df.shape[0])
df = df[~df.duplicated(keep='first')]
print("after", df.shape[0])

before 1357438
after 1357383


Then, to deal with the duplicates on differents subsets of the columns (['code'] and key_cols), we are going to define a function that combines all the duplicated rows in one. The main row will be the one that has the greater number of unique not null values, and it will be unriched by date from the others rows if available (using pd.DataFrame.combine_first).

In [0]:
# Function combining all dupl. rows,
# '1st' is the one with highest unique notna val.
''' Among each group of duplicates :
 - finds nb of unique notna for each row
 - selects the one with the most unique notna
 - combines the rows (1st is the selected row)
 - replaces all the rows by combined row
 - returns nb of grps of dupl., indexes of rows to drop, list of rows for update
 '''
def comb_dup_rows(df, cols):
  uniq_notna = lambda x:pd.notna(x.unique()).sum()
  df_f = pd.DataFrame({})
  l_ind = []
  cpt = 0
  for k,df_g in df.groupby(by=cols): # loop on all grps of dup.
    df_comb = pd.DataFrame({})
    ser_gr = df_g.apply(uniq_notna, axis=1)
    idx_max = np.argmax(ser_gr)
    ind = list(ser_gr.index)
    ind.remove(idx_max)
    df_comb = df_g.loc[idx_max]
    for i in ind: # loop on all dupl. except that with max unique not na
      df_comb = df_comb.combine_first(df_g.loc[i])
    df_f = df_f.append(df_comb)
    l_ind += ind
    cpt += 1
  df_f.reindex(columns=df.columns) #reorder the columns (bug in pd.concat)
  return cpt, l_ind, df_f 

Let's apply this function to dfcopy, with the subset ['code'] for the duplicates:

In [0]:
# 2 - Checking for duplicates on 'code'
df_dup = dfcopy[dfcopy.duplicated(subset=['code'],keep=False)]
print("Nb of duplicated rows on 'code':", df_dup.shape[0])

In [0]:
# duplicates on 'code' column
# -> Keeping the combined row, deleting the others
print("Shape of dfcopy, before", df_dup.shape[0])
nb_gr, ind_drop, df_dup_f = comb_dup_rows(df_dup, ['code'])
df_dup.drop(ind_drop, inplace=True)
df_dup.update(df_dup_f, overwrite=True)
print("Nb of groups of duplicates :", nb_gr)
print("Nb of duplicates :", df_dup.shape[0])
print("Nb of rows to delete:", len(ind_drop))
print("Shape of dfcopy, after", df_dup.shape[0])

In [0]:
# Changing dfcopy
dfcopy.drop(ind_drop, inplace=True)
dfcopy.update(df_dup_f, overwrite=True)

In [0]:
speak('dropping duplicates (on column code)')

29 duplicates has be deleted, and 29 rows updated.

Let's apply this function to dfcopy, with the subset key_cols for the duplicates:

In [0]:
dfcopy.groupby(['code_val']).size()

In [0]:
# 3 - Checking for duplicates on key_cols
key_cols = ['product_name', 'quantity_num', 'quantity_unit', 'brands']
m_dup_keys = dfcopy.duplicated(subset=key_cols,keep=False)
m_c_inval = dfcopy['code_val']==False
df_dup_k_c_val = dfcopy[m_dup_keys & ~m_c_inval]
df_dup_k_c_inval= dfcopy[m_dup_keys & m_c_inval]
print(f"Nb of duplicated rows on key_cols with VALID code: {df_dup_k_c_val.shape[0]}" )
print(f"Nb of duplicated rows on key_cols with INVALID code: {df_dup_k_c_inval.shape[0]}")

If there is duplicates with different code, and at least one of them is valide and one invalid, we want to keep only the rows with valide codes:

In [0]:
# Finding duplicates with at least 1 row with valid code AND 1 invalid code
df_mixed_dup = dfcopy[m_dup_keys].groupby(by=key_cols)\
                .filter(lambda x:sum((x['code_val']==True).values)>1\
                        and sum((x['code_val']==False).values)>1)

In [0]:
# df_mixed_dup.sort_values(by=key_cols)
gb = df_mixed_dup.groupby(by=key_cols)
print(f"Nb of rows with mixed val/inval codes: {sum(gb.size())}")
print(f"Nb of groups of duplicates: {len(gb)}")

In [0]:
# shows example of duplicates on key_cols with mixed 'code_val'
gb.get_group(list(gb.groups.keys())[0])

In [0]:
# Dropping the rows with invalid codes in each group
print("Shape of dfcopy, before", dfcopy.shape[0])
for i in range(len(gb)):
  d = gb.get_group(list(gb.groups.keys())[i])
  ind = d[d['code_val']==False].index
  dfcopy.drop(index=ind, inplace=True)
print("Shape of dfcopy, after", dfcopy.shape[0])

In [0]:
# Refreshing the list
m_dup_keys = dfcopy.duplicated(subset=key_cols,keep=False)
m_c_inval = dfcopy['code_val']==False
df_dup_k_c_val = dfcopy[m_dup_keys & ~m_c_inval]
df_dup_k_c_inval= dfcopy[m_dup_keys & m_c_inval]
print(f"Nb of duplicated rows on key_cols with VALID code: {df_dup_k_c_val.shape[0]}" )
print(f"Nb of duplicated rows on key_cols with INVALID code: {df_dup_k_c_inval.shape[0]}")

In [0]:
# Duplicates on key_cols column with invalid codes
# -> Keeping the combined row, deleting the others
print("Shape of df_dup_k_c_inval, before", df_dup_k_c_inval.shape[0])
nb_gr, ind_drop, df_dup_f = comb_dup_rows(df_dup_k_c_inval, key_cols)
df_dup_k_c_inval.drop(ind_drop, inplace=True)
df_dup_k_c_inval.update(df_dup_f, overwrite=True)
print("Nb of groups of duplicates :", nb_gr)
print("Nb of duplicates :", df_dup_k_c_inval.shape[0])
print("Nb of rows to delete:", len(ind_drop))
print("Shape of df_dup_k_c_inval, after", df_dup_k_c_inval.shape[0])

In [0]:
# Changing dfcopy
dfcopy.drop(ind_drop, inplace=True)
dfcopy.update(df_dup_k_c_inval, overwrite=True)

In [0]:
# list(df.columns[df.columns.str.contains('nutri')]) 'nutriscore_score', 'nutrition-score-fr_100g'

In [0]:
# Exporting the dataframe as CSV for further treatment
df.to_csv('OFF_up_to_1_3.csv')